## Install YOLOv8

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 10.8 MB/s eta 0:00:00


In [2]:
!pip install ultralytics albumentations opencv-python-headless

## Mount your Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Navigate to the location

In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset

/content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset


## Import necessary libraries

In [5]:
from ultralytics import YOLO
import albumentations as A
import cv2
import os
from glob import glob
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Define augmentation function

In [6]:
def augment_and_save(image_dir, label_dir, output_image_dir, output_label_dir, num_augments=3):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    # Define augmentation pipeline
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.Rotate(limit=30, p=0.3),
        A.RandomScale(scale_limit=0.2, p=0.3),
        A.Blur(blur_limit=3, p=0.2),
    ])

    # Loop through images in the directory
    for image_path in glob(f"{image_dir}/*.jpg"):
        img = cv2.imread(image_path)
        base_name = os.path.basename(image_path)

        # Save the original image
        cv2.imwrite(os.path.join(output_image_dir, base_name), img)

        # Copy the corresponding label file
        label_name = os.path.splitext(base_name)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(output_label_dir, label_name))

        # Create augmented images
        for i in range(num_augments):
            augmented = transform(image=img)
            aug_img = augmented["image"]
            aug_name = f"{os.path.splitext(base_name)[0]}_aug_{i}.jpg"
            cv2.imwrite(os.path.join(output_image_dir, aug_name), aug_img)
            aug_label_name = f"{os.path.splitext(base_name)[0]}_aug_{i}.txt"
            shutil.copy(label_path, os.path.join(output_label_dir, aug_label_name))

# Apply augmentation to training images
augment_and_save(
    image_dir='train/images',
    label_dir='train/labels',
    output_image_dir='train_augmented/images',
    output_label_dir='train_augmented/labels',
    num_augments=3
)

## Resize validation and test images

In [7]:
def resize_images_with_labels(image_dir, label_dir, output_image_dir, output_label_dir, target_size=(640, 640)):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    for image_path in glob(f"{image_dir}/*.jpg"):  # Adjust extension as needed
        img = cv2.imread(image_path)
        resized_img = cv2.resize(img, target_size)

        # Save the resized image
        image_name = os.path.basename(image_path)
        cv2.imwrite(os.path.join(output_image_dir, image_name), resized_img)

        # Copy the corresponding label file
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(output_label_dir, label_name))

# Resize validation and test sets
resize_images_with_labels(
    image_dir='valid/images',
    label_dir='valid/labels',
    output_image_dir='val_resized/images',
    output_label_dir='val_resized/labels',
    target_size=(640, 640)
)

resize_images_with_labels(
    image_dir='test/images',
    label_dir='test/labels',
    output_image_dir='test_resized/images',
    output_label_dir='test_resized/labels',
    target_size=(640, 640)
)

## Update dataset YAML file

In [8]:
dataset_yaml = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/data.yaml'
with open(dataset_yaml, 'w') as f:
    f.write(f"""
train: /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/train_augmented/images
val: /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/val_resized/images
test: /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/test_resized/images
nc: 1
names: ['Pothole']
""")

## Initialize YOLO model

In [9]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 196MB/s]


## Train the YOLO model

In [10]:
model.train(
    data=dataset_yaml,
    epochs=50,               # Number of training epochs
    imgsz=640,               # Image size
    batch=16,                # Batch size
    workers=2                # Number of workers
)

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

100%|██████████| 755k/755k [00:00<00:00, 125MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 198MB/s]


AMP: checks passed ✅


...

                   all         80        777      0.421      0.302      0.287      0.125



50 epochs completed in 0.543 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.57it/s]


                   all         80        777      0.434      0.363      0.344      0.148
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 6.4ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e8c864660b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

## Evaluate the model on validation set

In [11]:
metrics = model.val()
print(f"Evaluation Metrics: {metrics}")

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/val_resized/labels.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 673, len(boxes) = 777. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.11s/it]


                   all         80        777       0.43      0.359      0.343      0.148
Speed: 0.5ms preprocess, 7.1ms inference, 0.0ms loss, 16.4ms postprocess per image
Results saved to runs/detect/train2
Evaluation Metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e8c846af460>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031, 

## Predict on the test set and save results

In [12]:
results = model.predict(
    source='/content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/test_resized/images',
    save=True
)


image 1/181 /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/test_resized/images/438_jpg.rf.a285aa02fc3f286de8b11b8704967079.jpg: 640x640 3 Potholes, 7.2ms
image 2/181 /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/test_resized/images/444_jpg.rf.0efbfcdab7d6220563701538ce327dae.jpg: 640x640 16 Potholes, 14.1ms
image 3/181 /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/test_resized/images/447_jpg.rf.d8ff97fede18d6ad54595876348bff81.jpg: 640x640 10 Potholes, 7.3ms
image 4/181 /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/test_resized/images/455_jpg.rf.c20af0d25997939d58a071d014231f67.jpg: 640x640 (no detections), 7.2ms
image 5/181 /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/test_resized/images/458_jpg.rf.d3385df94d4fb6f09d08e16db8fc3336.jpg: 640x640 2 Potholes, 7.3ms
image 6/181 /content/drive/MyDrive/Colab Notebooks/Computer_V

## Perform additional training for refinement

In [13]:
model.train(
    data=dataset_yaml,
    epochs=25,
    batch=16
)

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

train: Scanning /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/train_augmented/labels.cache... 1388 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1388/1388 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 7512, len(boxes) = 8708. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/Colab Notebooks/Computer_Vision_Project/Pothole_Dataset/val_resized/labels.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 673, len(boxes) = 777. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all         80        777      0.412        0.3      0.295      0.133



25 epochs completed in 0.291 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.14it/s]


                   all         80        777      0.466      0.332      0.336      0.148
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 6.4ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e8c864dc100>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

## Export the model to ONNX format for deployment

In [14]:
model.export(format='onnx')

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train4/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 140.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 262.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 209.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 243.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 20.2s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updat

'runs/detect/train4/weights/best.onnx'